In [2]:
from pymatgen.core import Lattice, Structure 
import numpy as np 
from pymatgen.core.composition import Element, Composition
from pymatgen.core.periodic_table import Specie
import math
import random
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Potcar,Incar
from pymatgen.io.vasp.outputs import Outcar 
from pymatgen.entries.computed_entries import ComputedStructureEntry
import os
import clease
from clease.settings import Concentration
import numpy as np
from clease.settings import CEBulk
from clease.structgen import NewStructures
from ase.db import connect
from ase.io import write
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter
import json
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer
from ase.neighborlist import NeighborList

# Functions 

In [3]:
def create_cca_primitive(atom_dict, a, prim = True):
    """
    Create a CCA (Complex Concentrated Alloy) primitive structure using the given atom dictionary and lattice constant.

    Parameters:
        atom_dict (dict): A dictionary containing the count of each atom in the supercell you eventually want.
        a (float): The lattice constant of the cubic cell.

    Returns:
        pymatgen.core.structure.Structure: The CCA supercell structure.

    """
    comp_list = list(atom_dict.keys())

    elements = [{element: count/sum(atom_dict.values())} for element, count in atom_dict.items()]
    c_v = 1 - sum(elements[0].values())
    elements[0][comp_list[0]] = c_v

    merged_elements = {k:v for element in elements for k,v in element.items()}
    elements = [merged_elements]
    #print(elements)
    direct_coords = [[0,0,0]]
    if prim:
        test_bcc = Structure(Lattice.cubic(a), comp_list, direct_coords)
    else:
        test_bcc = Structure.from_spacegroup("Im-3m",Lattice.cubic(a),elements,direct_coords)
    return test_bcc

def closest_composition(comp, num_atoms):
    # Calculate the total fraction
    total_fraction = sum(comp.values())
    
    # Calculate the number of atoms for each element
    atoms = {element: fraction / total_fraction * num_atoms for element, fraction in comp.items()}
    
    # Round up non-integer atom counts and adjust the first element to balance
    elements = list(atoms.keys())
    for i in range(1, len(elements)):
        if atoms[elements[i]] % 1 != 0:
            atoms[elements[i]] = int(atoms[elements[i]]) + 1
            #num_atoms += 1

    atoms[elements[0]] = num_atoms - sum(atoms[elements[i]] for i in range(1, len(elements)))
    
    # Calculate the actual fractions
    actual_fractions = {element: num / num_atoms for element, num in atoms.items()}
    
    return atoms, actual_fractions

def create_random_supercells(composition,alat,supercell_size,db_name,num_structures):
    # A_eq is the identity matrix with a size equal to the number of elements in the composition, i.e len(composition.keys())
    A_eq = np.eye(len(composition.keys()))
    b_eq = list(composition.values())
    conc = Concentration(basis_elements=[list(composition.keys())],
                         A_eq = A_eq,
                         b_eq = b_eq)
    
    settings = CEBulk(crystalstructure='bcc',
                      a=alat,
                      size=[supercell_size,supercell_size,supercell_size],
                      concentration=conc,
                      db_name=db_name,
                      max_cluster_dia=[6.0, 4.5, 4.5])
    


    ns = NewStructures(settings, generation_number=1,
                    struct_per_gen=num_structures)
    ns.generate_random_structures()

def connect_ase_db(db_name):
    from ase.db import connect
    atoms = connect(db_name)
    print(len(atoms))
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)
    return atoms
        
def ase_db_to_pymatgen(db_name,distance = 0.1, output_file=None):

    data = {}
    Adaptor = AseAtomsAdaptor()
    atoms = connect(db_name) 

    for idx in range(2,len(atoms)+1):
        
        curr_struct = atoms.get(id=idx).toatoms()
        
        curr_pymatgen = Adaptor.get_structure(curr_struct)
        
        
        trans = PerturbStructureTransformation(distance=distance, min_distance=0.005)
        distorted_struct = trans.apply_transformation(curr_pymatgen)
        
        data.update({f'{idx}':distorted_struct.as_dict()})
        
    if output_file is not None:
        with open(output_file, 'w') as f:
            json.dump(data, f,)
    else:
        return data
        
def generate_random_supercells(composition, num_structures, lattice_parameter=3.01, supercell_size=4,supercell_type='cubic', seed=42):
    random.seed(seed) 
    supercells = []

    comp_list = [key for key in composition]
    for _ in range(num_structures):
        # Create a bulk V crystal with the specified lattice parameter
        #prim_cell = bulk(comp_list[0], cubic=True, a=lattice_parameter)
        if supercell_type == 'cubic':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0],comp_list[0]], [[0, 0, 0],[0.5, 0.5, 0.5]])
        elif supercell_type == 'prim':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0]], [[0, 0, 0]])
        
        # make the supercell
        supercell = prim_cell * (supercell_size,supercell_size,supercell_size)

        # Create a list of all possible indices for the current element
        all_indices = list(range(len(supercell.sites)))

        for element, count in composition.items():
            # Randomly select 'count' indices for the current element
            selected_indices = random.sample(all_indices, count)
            
            # Change the selected indices to the current element
            for index in selected_indices:
                supercell.replace(index,element)
            
            # remove the selected_indices from the all_indices list
            all_indices = [index for index in all_indices if index not in selected_indices]
        
        supercell = supercell.get_sorted_structure()

        supercells.append(supercell)

    return supercells

def make_vasp_job(supercell, job_path, kpoints_params, incar_params=None):
    """
    Create a VASP job for the given supercell.

    Parameters:
        supercell (pymatgen.core.structure.Structure): The supercell for which to create the VASP job.
        job_path (str): The path to the directory where the VASP job should be created.
        incar_params (dict): A dictionary containing the INCAR parameters.
        kpoints_params (list): A tuple containing the kpoints parameters i.e (3,3,3)
        potcar_path (str): The path to the directory containing the POTCAR files.

    Returns:
        None

    """
    # make the POSCAR 
    poscar = Poscar(structure = supercell, comment = str(supercell.composition))
    
    # make the KPOINTS
    kpoints = Kpoints.gamma_automatic(kpts = kpoints_params)
    
    # make the POTCAR
    #potcar = Potcar(symbols=[str(e.symbol)+"_pv" for e in supercell.composition.elements], functional="PBE_64")

    symbols = []
    for e in supercell.composition.elements:
        try:
            symbols.append(str(e.symbol) + "_pv")
            potcar = Potcar(symbols=symbols, functional="PBE_64")
        except:
            symbols[-1] = str(e.symbol) + "_sv"
            potcar = Potcar(symbols=symbols, functional="PBE_64")
    
    potcar = Potcar(symbols=symbols, functional="PBE_64")
    
    # make the INCAR
    if incar_params is None:
        incar_params = {'ALGO':'Fast',
              'ENCUT': 360,
              'EDIFF': 1E-6,
              'EDIFFG': -0.01,
              'ISMEAR': 1,
              'SIGMA': 0.2,
              'LWAVE': '.FALSE.',
              'LCHARG': '.FALSE.',
              'LORBIT': 11,
              'LREAL': '.FALSE.',
              'NELM' : 100,
              'NELMIN' : 5,
              'NSW' : 200,
              'PREC' : 'Accurate',
              'IBRION' : 3,
              'SMASS' : 1.85,
              'POTIM' : 0.25,
              'ISIF' : 3,
              #'NBANDS' : supercell.num_sites*2*5, # assuming 5 electrons per atom
              }
    else:
        incar_params = incar_params
    incar = Incar(incar_params)

    # make slurm file 
    
    # write the files to the job_path
    poscar.write_file(filename=os.path.join(job_path, "POSCAR"))
    kpoints.write_file(filename=os.path.join(job_path, "KPOINTS"))
    potcar.write_file(filename=os.path.join(job_path, "POTCAR"))
    incar.write_file(filename=os.path.join(job_path, "INCAR"))

def make_slurm_file(job_path, i, time="24:00:00", nodes=1, num_gpus=1):
    """
    Create a SLURM job for the given VASP job.

    Parameters:
        job_path (str): The path to the directory containing the VASP job.
        job_name (str): The name of the job.
        time (str): The time the job should run for.
        nodes (int): The number of nodes the job should run on.
        ppn (int): The number of processors per node.
        queue (str): The queue to which the job should be submitted.

    Returns:
        None

    """
    slurm_file = open(os.path.join(job_path, "job.slurm"), "w")
    slurm_file.write("#!/bin/bash\n")
    slurm_file.write(f"#SBATCH --job-name=job_{i}\n")
    slurm_file.write("#SBATCH --output=std-out\n")
    slurm_file.write("#SBATCH --time=" + time + "\n")
    slurm_file.write("#SBATCH --nodes=" + str(nodes) + "\n")
    slurm_file.write(f"#SBATCH --ntasks-per-node={num_gpus}\n")
    slurm_file.write("#SBATCH --constraint=v100s\n")
    slurm_file.write(f"#SBATCH --gres=gpu:{num_gpus}\n")
    #slurm_file.write("#SBATCH --partition=" + queue + "\n")
    #slurm_file.write("module load vasp/5.4.4\n")
    slurm_file.write("export ")
    slurm_file.write("cd $SLURM_SUBMIT_DIR\n")
    slurm_file.write("/opt/nvidia/hpc_sdk/Linux_x86_64/22.5/comm_libs/mpi/bin/mpirun /home/myless/VASP/vasp.6.3.2/bin/vasp_std\n")
    slurm_file.write("exit\n")
    #slurm_file.write("mpirun vasp_std\n")
    slurm_file.close()
    
    #!/bin/bash # #SBATCH --job-name=neb_Cr9Ti10V44_Structure_Num_0_Vac_site_num_0_110 #SBATCH --output=std-out #SBATCH --ntasks-per-node=3 #SBATCH --nodes=1 #SBATCH --gres=gpu:3 #SBATCH --constraint=v100s #SBATCH --time=24:00:00 #SBATCH -p regular

def create_computed_entry_from_outcar(outcar):
    """
    Create a ComputedEntry from the given OUTCAR file."""
    # need to test this with an outcar 
    outcar = Outcar(outcar)
    energy = outcar.final_energy
    composition = outcar.structure.composition
    structure = outcar.structure

    return ComputedStructureEntry(composition, energy, structure = structure)

def create_computed_entry_from_chgnet(structure, energy):
    composition = structure.composition
    return ComputedStructureEntry(composition=composition, energy = energy, structure=structure)
    
def relax_structures(supercells, potential_path,device='cpu',verbose=False):
    vac_pot = CHGNet.from_file(potential_path, use_device=device)
    entries = []
    relaxer = StructOptimizer(vac_pot, use_device=device)

    # If vcrti_generated_supercells is a dictionary, convert it to a list
    if isinstance(supercells, dict):
        supercells = list(supercells.values())
    print(len(supercells))
    for i, supercell in enumerate(supercells):
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
        relaxed_supercell = relaxer.relax(atoms=supercell, fmax=0.05, relax_cell=True, verbose=verbose)
        print("Finished Volumetric Relaxing Structure: ", i)
        final_result = relaxer.relax(atoms=relaxed_supercell['final_structure'], fmax=0.05, relax_cell=False, verbose=verbose)
        print("Final Energy: ", final_result['trajectory'].energies[-1])

        # make the computed entry
        entry = create_computed_entry_from_chgnet(supercell, final_result['trajectory'].energies[-1])
        entries.append(entry.as_dict())

    return entries

def return_x_neighbors(atoms, target_atom_index, x_neighbor, alat, tolerance = 1.05):
    """
    Returns the indices and distances of the nearest neighbors of a target atom.
    
    Parameters:
    atoms (Atoms): The atomic structure.
    target_atom_index (int): The index of the target atom.
    x_neighbor (int): The number of neighbors to return (1, 2, or 3).
    alat (float): The lattice constant.
    tolerance (float, optional): The tolerance factor for the cutoff distance. Default is 1.05.
    
    Returns:
    nearest_neighbors (list): The indices of the nearest neighbors.
    distances (list): The distances to the nearest neighbors.
    """

    # get the cutoff distance for the nearest neighbors
    if x_neighbor == 1:
        cutoff = alat * np.sqrt(3)/2 * tolerance # 5% larger than the nearest neighbor distance for tolerance
    elif x_neighbor == 2:
        cutoff = alat * tolerance 
    elif x_neighbor == 3:
        cutoff = alat * np.sqrt(2) * tolerance
    else:
        print('x_neighbor must be 1, 2, or 3')
        return None
    
    # create the neighbor list 
    nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
    nl.update(atoms)
    indices, offsets = nl.get_neighbors(target_atom_index)
    distances = [atoms.get_distance(target_atom_index, i, mic=True) for i in indices]

    # get the number of indices based on if we need the nearest, next-nearest, or next-next-nearest neighbors
    if x_neighbor == 1:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[:8]
    elif x_neighbor == 2:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[8:14]
    elif x_neighbor == 3:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[14:26]
    else:
        print('x_neighbor must be 1, 2, or 3')
        return None
    
    # get the indices and distances of the nearest neighbors sorted by distance
    nearest_neighbors = [index for index,_ in sorted_neighbors]
    distances = [distance for _, distance in sorted_neighbors]
    return nearest_neighbors, distances

# Run the Code

## create the db and the atom_dict

In [24]:
x_cr = 0.02
x_ti  = 0.02
x_w = 0.04
x_zr  = 0.01
x_ta = 0.005
x_v = 1 - x_cr - x_ti - x_w - x_zr - x_ta
supercell_size = 5
num_atoms = supercell_size**3 
num_cr  = math.ceil(num_atoms * x_cr)
num_ti = math.ceil(num_atoms * x_ti)
num_w  = math.ceil(num_atoms * x_w)
num_zr = math.ceil(num_atoms * x_zr)
num_ta = math.ceil(num_atoms * x_ta)
#num_v = 2*supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
num_v = supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti , 'Zr' : num_zr, 'W' : num_w, 'Ta' : num_ta}
#atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti}
num_structures = 30 
print(atom_dict)
db_name = '_'.join([f'{key}{value}' for key, value in atom_dict.items()]) + '.db'
print(db_name)


{'V': 111, 'Cr': 3, 'Ti': 3, 'Zr': 2, 'W': 5, 'Ta': 1}
V111_Cr3_Ti3_Zr2_W5_Ta1.db


In [25]:
from ase.visualize import view
db_name = 'V111_Cr3_Ti3_Zr2_W5_Ta1.db'
test_atoms = connect_ase_db(db_name)

31
Atoms(symbols='V', pbc=True, cell=[[-1.505, 1.505, 1.505], [1.505, -1.505, 1.505], [1.505, 1.505, -1.505]], tags=...)
Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)
Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)
Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)
Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.524999999

In [22]:
first = test_atoms.get(id=2).toatoms()
view(first, viewer='x3d')

## Generate Supercells

In [4]:
new_atoms, new_atom_dict = closest_composition(atom_dict, num_atoms)

In [26]:
generated_supercells = create_random_supercells(new_atom_dict, 3.01, supercell_size, db_name, num_structures)
# I want the db name to be the composition of the supercell coming from atom dict, i.e key0value0_key1value1_key2value2_keynvaluen.db

In [27]:
generated_structures = ase_db_to_pymatgen(db_name=db_name)

## Visualize Supercell

In [33]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

print(AseAtomsAdaptor.get_atoms(Structure.from_dict(generated_structures['4'])))

Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)


In [29]:
import json 
# write all the structures to a json 
structures = []
structures_name = 'structures_' + db_name.split('.')[0] + '.json'
with open(structures_name, 'w') as f:
    for i, supercell in enumerate(generated_structures.values()):
        if isinstance(supercell, dict):
            structures.append(supercell)
        else:
            structures.append(supercell.as_dict())
    json.dump(structures, f)
        #f.write(supercell.to(fmt='poscar'))
        #f.write('\n')

In [32]:
structures_test = json.load(open(structures_name, 'r'))
print(AseAtomsAdaptor.get_atoms(Structure.from_dict(structures_test[0])))

Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)


## Relax the structures and Make VASP Jobs for CE

In [ ]:
pot_path = '/home/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'
entries = relax_structures(generated_structures, pot_path, device='cuda', verbose=True)

In [ ]:
#main_vasp_job_path = '/Users/myless/Dropbox (MIT)/Research/2024/IAP_2024/Structure_Making/optimized_clease_vcrti_structures/'
main_vasp_job_path = './test_gpu_mem/'
# make the directory for the jobs if it exists already, remove
if os.path.exists(main_vasp_job_path):
    import shutil
    shutil.rmtree(main_vasp_job_path)
os.makedirs(main_vasp_job_path, exist_ok=True)
for i, supercell in enumerate(generated_structures.values()):
    job_path = os.path.join(main_vasp_job_path, f'job_{i}')
    os.makedirs(job_path, exist_ok=True)
    if isinstance(supercell, dict):
        supercell = Structure.from_dict(supercell)
        supercell = supercell.get_sorted_structure()
    make_vasp_job(supercell, job_path, kpoints_params=(3,3,3), incar_params=None)
    make_slurm_file(job_path, i, num_gpus=1)
    print(f'Job {i} made')

## Create Defects using Nearest Neighbors

In [13]:
db_name = 'V111_Cr3_Ti3_Zr2_W5_Ta1.db'
#db_name = 'v4cr4ti.db'
structure_load = ase_db_to_pymatgen(db_name=db_name, distance=0.05, output_file=None)

In [14]:
structures = []
for i,struc in enumerate(structure_load.values()):
    if isinstance(struc, dict):
        structures.append(Structure.from_dict(struc))
    else:
        structures.append(struc)

In [15]:
defect_atoms = AseAtomsAdaptor.get_atoms(structures[0].get_sorted_structure())
target_atom_index = 6
n_neighbor = return_x_neighbors(atoms = defect_atoms,
                                target_atom_index= target_atom_index,
                                x_neighbor= 1,
                                alat=  3.01)
print(n_neighbor)

([107, 0, 120, 8, 7, 9, 25, 56], [2.573931470184882, 2.5800282141124207, 2.6012588414036633, 2.603164681748325, 2.611114716361887, 2.624244307381311, 2.6267861065027454, 2.646027747278684])


In [16]:
#print(defect_atoms[2].position)

def create_vac_defect(atoms, start_idx, end_idx):
    start_pos = atoms[start_idx].position
    end_pos = atoms[end_idx].position

    # copy the structures
    start_atoms = atoms.copy()
    end_atoms = atoms.copy()
    
    # move the atom to the start position
    end_atoms.positions[end_idx] = start_pos  

    del start_atoms[start_idx]
    del end_atoms[start_idx]

    return start_atoms, end_atoms
     

def get_position(atoms, index):
    return atoms[index].position

def print_atom_positions(atoms):
    # Print positions of each atom
    for i, atom in enumerate(atoms):
        print(f"Atom {i}: {atom.position}")

start_idx = target_atom_index 
end_idx = n_neighbor[0][0] 
end_pos = get_position(defect_atoms, end_idx)
start_pos = get_position(defect_atoms, start_idx)

print(f"Position of atom : {start_idx} =  {start_pos}")
print(f"Position of atom : {end_idx} =  {end_pos}")



Position of atom : 6 =  [9.01202072 6.00092944 5.99947121]
Position of atom : 107 =  [7.53247351 4.4869348  4.53527057]


In [17]:
from ase import Atoms
import numpy as np

def compare_positions(atoms1, atoms2):
    if len(atoms1) != len(atoms2):
        raise ValueError("The two Atoms objects must have the same number of atoms.")
    
    for i, (atom1, atom2) in enumerate(zip(atoms1, atoms2)):
        if not np.allclose(atom1.position, atom2.position):
            print(f"Atom {i} does not match:")
            print(f"Position in Start: {atom1.position}")
            print(f"Position in End: {atom2.position}")

print(f"Position of atom : {start_idx} =  {start_pos}")
print(f"Position of atom : {end_idx} =  {end_pos}")


Position of atom : 6 =  [9.01202072 6.00092944 5.99947121]
Position of atom : 107 =  [7.53247351 4.4869348  4.53527057]


In [18]:
from ase.io import write
start_atoms, end_atoms = create_vac_defect(defect_atoms, start_idx, end_idx)
main_dir = '/home/myless/Packages/structure_maker/poscar_position_125'

# check if main_dir exists, if it does not create, if it does, remove and create
if os.path.exists(main_dir):
    import shutil
    shutil.rmtree(main_dir)
os.makedirs(main_dir, exist_ok=True)


write(os.path.join(main_dir, "START"), start_atoms, format='vasp')
write(os.path.join(main_dir, "END"),end_atoms, format='vasp')


In [20]:
from ase.io import read
main_path = '/home/myless/Packages/structure_maker/poscar_position_125/neb_run/neb_Cr3Ti3V57_0_0_111'
cif_path = '/home/myless/Packages/structure_maker/poscar_position_125/cifs'

if not os.path.exists(cif_path):
    os.makedirs(cif_path, exist_ok=True)

# loop through folders
for i in range(7):
    atoms = read(os.path.join(main_path,f'0{i}','POSCAR'),format='vasp')
    write(os.path.join(cif_path,f'POSCAR_0{i}.cif'),atoms,format='cif')


    

# Graveyard

In [36]:
potential_path = '/Users/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'

entries = relax_structures(generated_supercells,potential_path=potential_path,device='mps',verbose=True)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps
30
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 14:59:28     -569.518066*       0.6284
FIRE:    1 14:59:29     -569.565369*       0.5979
FIRE:    2 14:59:29     -569.652222*       0.5363
FIRE:    3 14:59:30     -569.764282*       0.4536
FIRE:    4 14:59:30     -569.885315*       0.3780
FIRE:    5 14:59:31     -569.999634*       0.2887
FIRE:    6 14:59:32     -570.092468*       0.2117
FIRE:    7 14:59:32     -570.152466*       0.1229
FIRE:    8 14:59:32     -570.177246*       0.1001
FIRE:    9 14:59:33     -570.162842*       0.1690
FIRE:   10 14:59:33     -570.164185*       0.1666
FIRE:   11 14:59:33     -570.166626*       0.1618
FIRE:   12 14:59:33     -570.170166*       0.1540
FIRE:   13 14:59:34     -570.174561*       0.1429
FIRE:   14 14:59:34     -570.179504*       0.1287
FIRE:   15 14:59:35     -570.184753*       0.1115
FIRE:   16 14:59:35     -5

In [20]:
# write the entries 
with open(db_name.split('.')[0] +'_entries.json', 'w') as f:
    json.dump(entries, f)

In [6]:
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer

potential_path = '/Users/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'
#potential_path = '/home/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'
vac_pot = CHGNet.from_file(potential_path, use_device='mps')

#for i, supercell in enumerate(vcrti_generated_supercells):
#relaxer = StructOptimizer(vac_pot)
#relaxed_supercell = relaxer.relax(atoms = vcrti_generated_supercells[0], fmax = 0.01, relax_cell=True)
    #result = relaxer.relax(atoms = relaxed_supercell)

entries = []
relaxer = StructOptimizer(vac_pot,use_device='mps')
for i, supercell in enumerate(vcrti_generated_supercells):
    relaxed_supercell = relaxer.relax(atoms = supercell, fmax = 0.05, relax_cell=True,verbose=False)
    print("Finished Volumetric Relaxing Structure: ", i)
    final_result = relaxer.relax(atoms = relaxed_supercell['final_structure'], fmax = 0.05, relax_cell=False,verbose=False)
    print("Final Energy: ", final_result['trajectory'].energies[-1])

    # make the computed entry
    #entry = create_computed_entry_from_chgnet(final_result['final_structure'], final_result['trajectory'].energies[-1])
    entry = create_computed_entry_from_chgnet(supercell, final_result['trajectory'].energies[-1])
    entries.append(entry.as_dict())
    




CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps


KeyboardInterrupt: 

In [6]:
import json 
import shutil
with open('v4cr4ti_entries.json', 'w') as f:
    json.dump(entries, f)

source_folder = '/home/myless/Packages/structure_maker'
dest_folder = '/mnt/c/Users/myles/Dropbox (MIT)/Research/2024/IAP_2024/Structure_Making/'
shutil.copytree(source_folder, dest_folder)

FileExistsError: [Errno 17] File exists: '/mnt/c/Users/myles/Dropbox (MIT)/Research/2024/IAP_2024/Structure_Making/'

In [72]:
test_entry = create_computed_entry_from_chgnet(relaxed_supercell['final_structure'], relaxed_supercell['trajectory'].energies[-1])
print(test_entry.as_dict())

{'@module': 'pymatgen.entries.computed_entries', '@class': 'ComputedStructureEntry', 'energy': -564.9136962890625, 'composition': {'Ti': 3.0, 'V': 58.0, 'Cr': 3.0}, 'entry_id': None, 'correction': 0.0, 'energy_adjustments': [], 'parameters': {}, 'data': {}, 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[9.816883288100458, -0.045030822457039724, -0.2707692631636627], [-0.047158841067081375, 9.197235696296065, 0.05166922563444473], [-0.26635979526982545, 0.04595135277667574, 9.807948992369784]], 'pbc': (True, True, True), 'a': 9.820719997079205, 'b': 9.197501732448384, 'c': 9.811672767896699, 'alpha': 89.40193540084451, 'beta': 93.13672659512305, 'gamma': 90.56525454926621, 'volume': 884.8357032215285}, 'properties': {}, 'sites': [{'species': [{'element': 'Ti', 'occu': 1}], 'abc': [-0.010048356212177116, 0.024999448755645946, 0.16055000354673765], 'xyz': [-0.1425865512781129, 0.23775579807899946, 1.5786787336772603], 'prope

In [41]:
x_cr = 0.02
x_ti  = 0.03
x_w = 0.05
x_zr  = 0.02
x_ta = 0.01
x_v = 1 - x_cr - x_ti - x_w - x_zr - x_ta
supercell_size = 4
num_atoms = 2*supercell_size**3 
num_cr  = math.ceil(num_atoms * x_cr)
num_ti = math.ceil(num_atoms * x_ti)
num_w  = math.ceil(num_atoms * x_w)
num_zr = math.ceil(num_atoms * x_zr)
num_ta = math.ceil(num_atoms * x_ta)
num_v = 2*supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti , 'Zr' : num_zr, 'W' : num_w, 'Ta' : num_ta}
num_structures = 10 
print(atom_dict)
generated_supercells = generate_random_supercells(atom_dict, num_structures, supercell_size=supercell_size)


{'V': 109, 'Cr': 3, 'Ti': 4, 'Zr': 3, 'W': 7, 'Ta': 2}
